In [ ]:
! pip install bastionlab polars

In [1]:
from bastionlab import Connection, Identity
import polars as pl

In [2]:
data_owner = Identity.create("data_owner")

client = Connection("localhost", identity=data_owner).client

In [3]:
# load dataset
train_df = pl.read_csv(
    "diabetes_train.csv",
    sep=" ",
    has_header=False,
    new_columns=["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"],
)
test_df = pl.read_csv(
    "diabetes_test.csv",
    has_header=False,
    new_columns=["target"],
)

In [4]:
train_rdf = client.polars.send_df(train_df)
test_rdf = client.polars.send_df(test_df)

In [14]:
# Perform linear regression on `diabetes` dataset.
from bastionlab.linfa.trainers import LinearRegression

model = client.linfa.train(train_rdf, test_rdf, LinearRegression(), 0.85)
print(model)
res = client.linfa.predict(
    model, [60, 2, 28.2, 112.0, 185, 113.8, 42.0, 4.0, 4.9836, 93]
)
print(res.fetch())

LinearRegression(fit_intercept=True)
shape: (1, 1)
┌────────────┐
│ prediction │
│ ---        │
│ f64        │
╞════════════╡
│ 192.289151 │
└────────────┘
